In [154]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [155]:
# ACADEMY WINNING MOVIES ON IMDB SORTED DESC BOX OFFICE HTML
imdb_html = requests.get('https://www.imdb.com/search/title/?release_date=2010-01-01,&sort=boxoffice_gross_us,desc&count=250')
soup = BeautifulSoup(imdb_html.content, 'html.parser')

In [156]:
# Scraped through the html code to find the div with the movies
main_div_tag = soup.find('div', {'class':'lister-list'})

In [157]:
# returns all 250 movies per page
movies_per_page = main_div_tag.find_all('div', {'class':'lister-item mode-advanced'})

In [158]:
# returns list of contents from page (250)
contents = []
for item in movies_per_page:
    contents.append(item.find('div',{'class':'lister-item-content'}))

In [159]:
# looks through contents and retrieves movie id (imdb)
movie_ids =[]
for i in contents:
    movie_ids.append(i.find('h3').find('a')['href'].split('/')[2])

In [160]:
# retrieves movie titles
movie_titles =[]
for i in contents:
    movie_titles.append(str(i.find('h3').find('a')).split('>')[1].split('<')[0])

In [161]:
# to retrieve movie release year
years =[]
for i in contents:
    year = str(i.find('h3').find('span', {'class':'lister-item-year text-muted unbold'})).split('(')[-1].split(')')[0]
    years.append(int(year))

In [162]:
# retrieves rating
ratings =[]
for i in contents:
    ratings.append(str(i.find('p').find('span',{'class':'certificate'})).split('>')[1].split('<')[0])

In [163]:
# retrieves runtime
runtimes =[]
for i in contents:
    runtimes.append(str(i.find('p').find('span',{'class':'runtime'})).split('>')[1].split('<')[0].split()[0])

In [164]:
# retrieves genres and cleans it, creates list of genres for each movie
genres = []
for i in contents:
    clean_genre = []
    genres_unclean = str(i.find('p').find('span',{'class':'genre'})).split('>')[1].split('<')[0].split('\n')[1].split(',')
    for genr in genres_unclean:
        clean_genre.append(genr.strip())
    genres.append(clean_genre)   

In [173]:
# Finds imdb rating
imdb_rating = []
for i in contents:
    imdb_rating.append(float(i.find('div',{'class':'inline-block ratings-imdb-rating'})['data-value']))

In [192]:
# Finds metascore rating
meta_score_rating = []
for i in contents:
    score = str(i.find('div',{'class':'inline-block ratings-metascore'}).find('span')).split('>')[1].split('<')[0].strip()
    meta_score_rating.append(int(score))

In [255]:
# scraping votes
votes =[]
for i in contents:
    votes.append(int(str(i.find('p', {'class':'sort-num_votes-visible'}).find_all('span')[1]).split('>')[1].split('<')[0].replace(',','')))

In [264]:
# gross revenue
gross_rev =[]
for i in contents:
    gross_rev.append(int(str(i.find('p',{'class':'sort-num_votes-visible'}).find_all('span')[4]).split('"')[1].replace(',','')))

In [345]:
def get_text_from_a(a_tag):
    '''
    takes a 'a' tag argument from html and returns the text displayed
    '''
    return str(a_tag).split('>')[1].split('<')[0]

In [346]:
directors =[]
for i in contents:
    dirct =[]
    for i2 in i.find('p',{'class':''}):
        if i2 == i.find('p',{'class':''}).find('span'):
            break
        else:
            if i2 in i.find('p',{'class':''}).find_all('a'):
                dirct.append(get_text_from_a(i2))
    directors.append(dirct)

In [402]:
def get_stars(content):
    strz =[]
    for i2 in content.find('p',{'class':''}):
        if i2 in content.find('p',{'class':''}).find_all('a'):
            strz.append(get_text_from_a(i2))
        else:
            if i2 == content.find('p',{'class':''}).find('span'):
                strz = []
                continue
    return strz
     
    
stars = [get_stars(i) for i in contents] 

In [407]:
def creates_list_for_df(page):
    return [{'id':movie_ids,'title':movie_titles,     
    'year':years,'rated':ratings,'runtime':runtimes,
    'genre':genres,'imdb_rating':imdb_rating,
    'meta_score':meta_score_rating,
    'votes':votes,'revenue':gross_rev,
    'directors':directors,'stars':stars}]

In [405]:
page1 = pd.DataFrame({'id':movie_ids,'title':movie_titles,
              'year':years,'rated':ratings,'runtime':runtimes,
              'genre':genres,'imdb_rating':imdb_rating,
             'meta_score':meta_score_rating,
             'votes':votes,'revenue':gross_rev,
             'directors':directors,'stars':stars})

In [406]:
page1

,id,title,year,rated,runtime,genre,imdb_rating,meta_score,votes,revenue,directors,stars
0,tt2488496,Star Wars: Episode VII - The Force Awakens,2015,PG-13,138,"[Action, Adventure, Sci-Fi]",7.9,81,827678,936662225,[J.J. Abrams],"[Daisy Ridley, John Boyega, Oscar Isaac, Domhn..."
1,tt4154796,Avengers: Endgame,2019,PG-13,181,"[Action, Adventure, Drama]",8.5,78,675333,858373000,"[Anthony Russo, Joe Russo]","[Robert Downey Jr., Chris Evans, Mark Ruffalo,..."
2,tt1825683,Black Panther,2018,PG-13,134,"[Action, Adventure, Sci-Fi]",7.3,88,566410,700059566,[Ryan Coogler],"[Chadwick Boseman, Michael B. Jordan, Lupita N..."
3,tt4154756,Avengers: Infinity War,2018,PG-13,149,"[Action, Adventure, Sci-Fi]",8.5,68,750679,678815482,"[Anthony Russo, Joe Russo]","[Robert Downey Jr., Chris Hemsworth, Mark Ruff..."
4,tt0369610,Jurassic World,2015,PG-13,124,"[Action, Adventure, Sci-Fi]",7.0,59,557045,652270625,[Colin Trevorrow],"[Chris Pratt, Bryce Dallas Howard, Ty Simpkins..."
...,...,...,...,...,...,...,...,...,...,...,...,...
245,tt1133985,Green Lantern,2011,PG-13,114,"[Action, Adventure, Sci-Fi]",5.5,39,258750,116601172,[Martin Campbell],"[Ryan Reynolds, Blake Lively, Peter Sarsgaard,..."
246,tt6343314,Creed II,2018,PG-13,130,"[Drama, Sport]",7.1,66,92312,115715889,[Steven Caple Jr.],"[Michael B. Jordan, Sylvester Stallone, Tessa ..."
247,tt1809398,Unbroken,2014,PG-13,137,"[Biography, Drama, Sport]",7.2,59,136618,115637895,[Angelina Jolie],"[Jack O'Connell, Miyavi, Domhnall Gleeson, Gar..."
248,tt5117670,Peter Rabbit,2018,PG,95,"[Adventure, Comedy, Family]",6.5,51,31441,115253424,[Will Gluck],"[James Corden, Fayssal Bazzi, Domhnall Gleeson..."
